In [2]:
import random
import shutil
from pathlib import Path
from collections import defaultdict
import yaml
from tqdm import tqdm
import numpy as np

In [3]:
def create_balanced_subset(
    dataset_root,
    output_root,
    train_samples=4000,
    val_samples=1000,
    negative_ratio=0.25,
    seed=42
):
    """
    Tạo subset cân bằng từ dataset gốc
    
    Args:
        dataset_root: Đường dẫn đến dataset gốc (dataset_output)
        output_root: Đường dẫn để lưu subset mới
        train_samples: Số lượng samples cho train (3500)
        val_samples: Số lượng samples cho val (500)
        negative_ratio: Tỉ lệ negative samples (0.25 = 25%)
        seed: Random seed
    
    Strategy "Cân bằng":
    - Perfectly balanced: Mỗi class có số lượng samples bằng nhau
    - Nếu một class không đủ samples, lấy tất cả và phân bổ lại cho các class khác
    - Negative samples được lấy ngẫu nhiên từ tất cả negative samples
    """
    random.seed(seed)
    np.random.seed(seed)
    
    dataset_root = Path(dataset_root)
    output_root = Path(output_root)
    
    # Load class names
    with open(dataset_root / 'data.yaml', 'r') as f:
        data = yaml.safe_load(f)
        class_names = data.get('names', [])
        num_classes = data.get('nc', len(class_names))
    
    print("=" * 70)
    print("📊 TẠO BALANCED SUBSET")
    print("=" * 70)
    print(f"Classes: {class_names}")
    print(f"Train samples: {train_samples} (25% negative = {int(train_samples * negative_ratio)})")
    print(f"Val samples: {val_samples} (25% negative = {int(val_samples * negative_ratio)})")
    print("=" * 70)
    
    # Tính toán số lượng samples cho mỗi class
    train_positive = int(train_samples * (1 - negative_ratio))  # 2625
    val_positive = int(val_samples * (1 - negative_ratio))      # 375
    train_negative = train_samples - train_positive              # 875
    val_negative = val_samples - val_positive                    # 125
    
    samples_per_class_train = train_positive // num_classes     # ~375
    samples_per_class_val = val_positive // num_classes          # ~53
    
    print(f"\n📈 Phân bổ:")
    print(f"Train - Positive: {train_positive} ({samples_per_class_train}/class)")
    print(f"Train - Negative: {train_negative}")
    print(f"Val - Positive: {val_positive} ({samples_per_class_val}/class)")
    print(f"Val - Negative: {val_negative}")
    
    # Tạo thư mục output
    for split in ['train', 'val']:
        (output_root / split / 'images').mkdir(parents=True, exist_ok=True)
        (output_root / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # Xử lý từng split
    for split in ['train', 'val']:
        print(f"\n{'='*70}")
        print(f"🔄 Xử lý {split.upper()}")
        print(f"{'='*70}")
        
        images_dir = dataset_root / split / 'images'
        labels_dir = dataset_root / split / 'labels'
        
        # Đọc tất cả images
        image_files = list(images_dir.glob('*.tif'))
        print(f"Tổng số images: {len(image_files)}")
        
        # Phân loại samples
        positive_by_class = defaultdict(list)  # {class_id: [image_paths]}
        negative_samples = []  # [image_paths]
        
        for img_path in tqdm(image_files, desc=f"Phân loại {split}"):
            label_path = labels_dir / (img_path.stem + '.txt')
            
            if not label_path.exists() or label_path.stat().st_size == 0:
                # Negative sample (không có label hoặc label rỗng)
                negative_samples.append(img_path)
            else:
                # Positive sample - đọc classes trong label
                with open(label_path, 'r') as f:
                    lines = f.readlines()
                    classes_in_image = set()
                    for line in lines:
                        if line.strip():
                            class_id = int(line.strip().split()[0])
                            classes_in_image.add(class_id)
                    
                    # Thêm vào tất cả classes có trong image
                    for class_id in classes_in_image:
                        positive_by_class[class_id].append(img_path)
        
        print(f"\n📊 Thống kê {split}:")
        print(f"Negative samples: {len(negative_samples)}")
        for class_id in range(num_classes):
            unique_samples = len(set(positive_by_class[class_id]))
            print(f"Class {class_id} ({class_names[class_id]}): {unique_samples} samples")
        
        # Sample cân bằng
        target_samples = train_samples if split == 'train' else val_samples
        target_positive = train_positive if split == 'train' else val_positive
        target_negative = train_negative if split == 'train' else val_negative
        samples_per_class = samples_per_class_train if split == 'train' else samples_per_class_val
        
        selected_images = []
        selected_labels = []
        
        # 1. Sample từ mỗi class (cân bằng)
        print(f"\n🎯 Sampling từ mỗi class ({samples_per_class}/class)...")
        class_counts = {}
        
        for class_id in range(num_classes):
            class_samples = list(set(positive_by_class[class_id]))  # Remove duplicates
            random.shuffle(class_samples)
            
            # Lấy samples_per_class hoặc tất cả nếu không đủ
            num_to_sample = min(samples_per_class, len(class_samples))
            selected = class_samples[:num_to_sample]
            
            class_counts[class_id] = len(selected)
            print(f"  Class {class_id} ({class_names[class_id]}): {len(selected)}/{len(class_samples)}")
            
            for img_path in selected:
                if img_path not in selected_images:  # Tránh duplicate
                    selected_images.append(img_path)
        
        # 2. Nếu thiếu samples, phân bổ lại từ các class còn dư
        current_positive = len(selected_images)
        if current_positive < target_positive:
            print(f"\n⚠️  Thiếu {target_positive - current_positive} positive samples, phân bổ lại...")
            remaining_needed = target_positive - current_positive
            
            # Tìm các class còn dư samples
            available_samples = []
            for class_id in range(num_classes):
                class_samples = list(set(positive_by_class[class_id]))
                already_selected = [img for img in selected_images if img in class_samples]
                remaining = [img for img in class_samples if img not in already_selected]
                available_samples.extend(remaining)
            
            random.shuffle(available_samples)
            additional = available_samples[:remaining_needed]
            selected_images.extend(additional)
            print(f"  Đã thêm {len(additional)} samples từ các class còn dư")
        
        # 3. Sample negative samples
        print(f"\n🎯 Sampling negative samples ({target_negative})...")
        random.shuffle(negative_samples)
        selected_negative = negative_samples[:target_negative]
        selected_images.extend(selected_negative)
        
        print(f"\n✅ Đã chọn {len(selected_images)} samples:")
        print(f"  - Positive: {len(selected_images) - len(selected_negative)}")
        print(f"  - Negative: {len(selected_negative)}")
        print(f"  - Phân bổ theo class:")
        for class_id in range(num_classes):
            count = sum(1 for img in selected_images[:len(selected_images)-len(selected_negative)]
                       if img in positive_by_class[class_id])
            print(f"    Class {class_id} ({class_names[class_id]}): {count}")
        
        # 4. Copy files
        print(f"\n📁 Copying files...")
        for img_path in tqdm(selected_images, desc=f"Copy {split}"):
            # Copy image
            dst_img = output_root / split / 'images' / img_path.name
            shutil.copy2(img_path, dst_img)
            
            # Copy label (nếu có)
            label_path = labels_dir / (img_path.stem + '.txt')
            if label_path.exists() and label_path.stat().st_size > 0:
                dst_label = output_root / split / 'labels' / label_path.name
                shutil.copy2(label_path, dst_label)
    
    # Copy data.yaml
    shutil.copy2(dataset_root / 'data.yaml', output_root / 'data.yaml')
    
    print(f"\n{'='*70}")
    print("✅ HOÀN THÀNH!")
    print(f"{'='*70}")
    print(f"Output: {output_root}")
    print(f"Train: {len(list((output_root / 'train' / 'images').glob('*.tif')))} samples")
    print(f"Val: {len(list((output_root / 'val' / 'images').glob('*.tif')))} samples")
    print(f"{'='*70}")

In [5]:


create_balanced_subset(
    dataset_root='/home/t1tc01-hoangphan/code/t1tc01-personal/ultralytics_sia-aero-eyes-Zalo-2025/dataset_output',
    output_root='/home/t1tc01-hoangphan/code/t1tc01-personal/ultralytics_sia-aero-eyes-Zalo-2025/dataset_output_balanced',
    train_samples=4000,
    val_samples=1000,
    negative_ratio=0.25,
    seed=42
)

📊 TẠO BALANCED SUBSET
Classes: ['Backpack', 'Jacket', 'Laptop', 'Lifering', 'Mobilephone', 'Person1', 'WaterBottle']
Train samples: 4000 (25% negative = 1000)
Val samples: 1000 (25% negative = 250)

📈 Phân bổ:
Train - Positive: 3000 (428/class)
Train - Negative: 1000
Val - Positive: 750 (107/class)
Val - Negative: 250

🔄 Xử lý TRAIN
Tổng số images: 18646


Phân loại train: 100%|██████████| 18646/18646 [00:06<00:00, 2873.08it/s]



📊 Thống kê train:
Negative samples: 3726
Class 0 (Backpack): 4638 samples
Class 1 (Jacket): 1852 samples
Class 2 (Laptop): 1871 samples
Class 3 (Lifering): 2645 samples
Class 4 (Mobilephone): 1857 samples
Class 5 (Person1): 2057 samples
Class 6 (WaterBottle): 0 samples

🎯 Sampling từ mỗi class (428/class)...
  Class 0 (Backpack): 428/4638
  Class 1 (Jacket): 428/1852
  Class 2 (Laptop): 428/1871
  Class 3 (Lifering): 428/2645
  Class 4 (Mobilephone): 428/1857
  Class 5 (Person1): 428/2057
  Class 6 (WaterBottle): 0/0

⚠️  Thiếu 432 positive samples, phân bổ lại...
  Đã thêm 432 samples từ các class còn dư

🎯 Sampling negative samples (1000)...

✅ Đã chọn 4000 samples:
  - Positive: 3000
  - Negative: 1000
  - Phân bổ theo class:
    Class 0 (Backpack): 587
    Class 1 (Jacket): 471
    Class 2 (Laptop): 480
    Class 3 (Lifering): 511
    Class 4 (Mobilephone): 472
    Class 5 (Person1): 479
    Class 6 (WaterBottle): 0

📁 Copying files...


Copy train: 100%|██████████| 4000/4000 [01:32<00:00, 43.40it/s] 



🔄 Xử lý VAL
Tổng số images: 6481


Phân loại val: 100%|██████████| 6481/6481 [00:02<00:00, 2373.44it/s]



📊 Thống kê val:
Negative samples: 1295
Class 0 (Backpack): 0 samples
Class 1 (Jacket): 0 samples
Class 2 (Laptop): 0 samples
Class 3 (Lifering): 0 samples
Class 4 (Mobilephone): 0 samples
Class 5 (Person1): 1129 samples
Class 6 (WaterBottle): 4057 samples

🎯 Sampling từ mỗi class (107/class)...
  Class 0 (Backpack): 0/0
  Class 1 (Jacket): 0/0
  Class 2 (Laptop): 0/0
  Class 3 (Lifering): 0/0
  Class 4 (Mobilephone): 0/0
  Class 5 (Person1): 107/1129
  Class 6 (WaterBottle): 107/4057

⚠️  Thiếu 536 positive samples, phân bổ lại...
  Đã thêm 536 samples từ các class còn dư

🎯 Sampling negative samples (250)...

✅ Đã chọn 1000 samples:
  - Positive: 750
  - Negative: 250
  - Phân bổ theo class:
    Class 0 (Backpack): 0
    Class 1 (Jacket): 0
    Class 2 (Laptop): 0
    Class 3 (Lifering): 0
    Class 4 (Mobilephone): 0
    Class 5 (Person1): 226
    Class 6 (WaterBottle): 524

📁 Copying files...


Copy val: 100%|██████████| 1000/1000 [00:21<00:00, 47.02it/s]


✅ HOÀN THÀNH!
Output: /home/t1tc01-hoangphan/code/t1tc01-personal/ultralytics_sia-aero-eyes-Zalo-2025/dataset_output_balanced
Train: 4000 samples
Val: 1000 samples
